# install stuff

In [4]:
! pip3 install pyspark pandas scikit-learn nltk catboost pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 116.9 MB/s eta 0:00:0000:0100:01


# setup pyspark

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import pyspark.sql
from pyspark.sql import functions as sf
import pyspark
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import isnan
from pyspark.sql.functions import dayofmonth, hour, month, weekofyear
from pyspark.sql.functions import length, col, log
from pyspark.sql.functions import lag
from pyspark.sql.types import DateType
from pyspark.sql.functions import min, max, mean, count, concat_ws, when
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import re
import unicodedata
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, pandas_udf
from pyspark.sql.types import StringType
import nltk
import numpy as np
nltk.download('stopwords')
from nltk.corpus import stopwords
from  pyspark.ml.feature import Tokenizer, Word2Vec
import pandas as pd
from pyspark.sql import DataFrame, Window
from pyspark.sql.functions import col, when, length, concat_ws, dayofmonth, hour, month, weekofyear, lag
from pyspark.sql.functions import min as Fmin, max as Fmax, mean as Fmean, count
from nltk.corpus import stopwords
from pyspark.sql.types import StringType
from pyspark.sql.functions import pandas_udf, lit
import pandas as pd
import unicodedata
# import catboost_spark
from typing import Type, TypeVar
from catboost import CatBoostRegressor, Pool
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# ss.stop()

In [4]:
sparkConf = pyspark.SparkConf() \
    .set("spark.executor.memory", "16g")\
    .set("spark.driver.memory", "16g")\
    .set("spark.driver.memoryOverheadFactor", "0.2")\
    .set("spark.driver.maxResultSize",  "0")\
    .set("spark.memory.fraction", "0.8") \
    .set("spark.memory.storageFraction", "0.2") \
    .set("spark.sql.shuffle.partitions", "200") \
    .set("spark.default.parallelism", "200")\
    .set("spark.executor.heartbeatInterval", "100s") \
    .set("spark.network.timeout", "600s") \
    .set("spark.rpc.askTimeout", "600s") \
    # .set("spark.jars.packages", "ai.catboost:catboost-spark_3.1_2.12:1.0.6")
ss = pyspark.sql.SparkSession.builder.config(conf=sparkConf).master('local[1]').getOrCreate()

24/06/15 17:13:03 WARN Utils: Your hostname, Olegs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.13 instead (on interface en0)
24/06/15 17:13:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/15 17:13:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
print(ss.sparkContext.uiWebUrl)

http://192.168.0.13:4040


# Feature maker classes

    1 балл за фичу, использующую метаданные каналов (макс 1 балл)
    2 балла за фичу, использующую текстовые поля (макс 2 балла)
    1 балл за фичу, использующую window aggregation, до 2 штук, различающихся по смыслу и реализации. Фичи которые можно альтернативно подсчитать групбаем и джойном не засчитываются (макс 2 балла)
    -1 балла за грязный код и -1 за отсутствие комментариев



0. не будем задумываться о временной структуре на трейне и будем считать эффект влияния просмотра предыдущего поста на пост через неделю малым
1. календарные фичи (из date)
    * День недели
    * Неделя месяца
    * Месяц
    * Час
2. Категориальные фичи
    * is_forwarded
    * has_image
3. **Числовые** (+1)
     * post_count * user_count
     * user_count * user_count
     * user_count / post_count
     * среднее число просмотров постов по группе!!! -> потом инфо с трейна сджоинить на тест и заимпутить средним наны
4. **Текстовые** (+2)
    * Длина text, description, name, title
    * Также сконкатим text, description, name, title и подадим в catboost! (изначально хотел попробовать spark_catboost, но, хоть там и можно категориальные обработать, но для текста нет фичи для спарка) Дополнительно я сделал udf функции для очистки текстов!

5. **Оконные** (+2)
    * Для каждой группы для каждого поста количество постов за прошедший месяц
    * -..- сколько часов прошло с предыдущего поста
    * среднее число просмотров за 10 последних постов в трейне (последний тренд)






In [5]:
class TextColProcessor:
    @pandas_udf(StringType())
    def cont_to_exp(text: pd.Series) -> pd.Series:
        """ Expand contractions in text """
        contractions = {
            "ain't": "am not",
            "aren't": "are not",
            "can't": "cannot",
            "can't've": "cannot have",
            "'cause": "because",
            "could've": "could have",
            "couldn't": "could not",
            "couldn't've": "could not have",
            "didn't": "did not",
            "doesn't": "does not",
            "don't": "do not",
            "hadn't": "had not",
            "hadn't've": "had not have",
            "hasn't": "has not",
            "haven't": "have not",
            "he'd": "he would",
            "he'd've": "he would have",
            "he'll": "he will",
            "he'll've": "he will have",
            "he's": "he is",
            "how'd": "how did",
            "how'd'y": "how do you",
            "how'll": "how will",
            "how's": "how does",
            "i'd": "i would",
            "i'd've": "i would have",
            "i'll": "i will",
            "i'll've": "i will have",
            "i'm": "i am",
            "i've": "i have",
            "isn't": "is not",
            "it'd": "it would",
            "it'd've": "it would have",
            "it'll": "it will",
            "it'll've": "it will have",
            "it's": "it is",
            "let's": "let us",
            "ma'am": "madam",
            "mayn't": "may not",
            "might've": "might have",
            "mightn't": "might not",
            "mightn't've": "might not have",
            "must've": "must have",
            "mustn't": "must not",
            "mustn't've": "must not have",
            "needn't": "need not",
            "needn't've": "need not have",
            "o'clock": "of the clock",
            "oughtn't": "ought not",
            "oughtn't've": "ought not have",
            "shan't": "shall not",
            "sha'n't": "shall not",
            "shan't've": "shall not have",
            "she'd": "she would",
            "she'd've": "she would have",
            "she'll": "she will",
            "she'll've": "she will have",
            "she's": "she is",
            "should've": "should have",
            "shouldn't": "should not",
            "shouldn't've": "should not have",
            "so've": "so have",
            "so's": "so is",
            "that'd": "that would",
            "that'd've": "that would have",
            "that's": "that is",
            "there'd": "there would",
            "there'd've": "there would have",
            "there's": "there is",
            "they'd": "they would",
            "they'd've": "they would have",
            "they'll": "they will",
            "they'll've": "they will have",
            "they're": "they are",
            "they've": "they have",
            "to've": "to have",
            "wasn't": "was not",
            " u ": " you ",
            " ur ": " your ",
            " n ": " and ",
            "won't": "would not",
            'dis': 'this',
            'bak': 'back',
            'brng': 'bring'
            }
        for key, value in contractions.items():
            text = text.str.replace(key, value)
        return text


    @pandas_udf(StringType())
    def remove_emails(text: pd.Series) -> pd.Series:
        """ Remove email addresses from text """
        return text.str.replace(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)', "", regex=True)


    @pandas_udf(StringType())
    def remove_urls(text: pd.Series) -> pd.Series:
        """ Remove URLs from text """
        return text.str.replace(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', regex=True)


    @pandas_udf(StringType())
    def remove_rt(text: pd.Series) -> pd.Series:
        """ Remove retweet indicators from text """
        return text.str.replace(r'\brt\b', '', regex=True).str.strip()


    @pandas_udf(StringType())
    def remove_special_chars(text: pd.Series) -> pd.Series:
        """ Remove special characters from text """
        text = text.str.replace(r'[^\w ]+', "", regex=True)
        text = text.str.split().str.join(" ")
        return text


    @pandas_udf(StringType())
    def remove_accented_chars(text: pd.Series) -> pd.Series:
        """ Remove accented characters from text """
        return text.apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore'))


    @pandas_udf(StringType())
    def remove_stopwords(text: pd.Series) -> pd.Series:
        """ Remove stopwords from text """
        # stopwords_list = stopwords.words("russian")
        # stopwords_list.extend(stopwords.words('english'))
        stopwords_list = ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
        return text.apply(lambda x: ' '.join([t for t in x.split() if t.lower() not in stopwords_list]))


    def remove_text_artifacts(df: DataFrame, text_col: str = 'combined_text') -> DataFrame:
        """ Apply a series of text cleaning functions """
        for text_proc_func in [
                                    TextColProcessor.cont_to_exp,
                                    TextColProcessor.remove_stopwords,
                                    TextColProcessor.remove_emails,
                                    TextColProcessor.remove_urls,
                                    TextColProcessor.remove_rt,
                                    TextColProcessor.remove_special_chars,
                                    ]:
            df = df.withColumn(text_col, text_proc_func(text_col))
        return df

In [6]:
class DataProcessor:
    """
    # Example:
    data_processor = DataProcessor()
    train_data, test_data, channel_data = data_processor.preprocess_data(train_data_sel, test_data_sel, channel_data_sel)

    mean_last_10_views_df = data_processor.compute_mean_last_10_views(train_data_sel)
    test_data = test_data.join(mean_last_10_views_df, 'channel_id', 'left')
    """

    intermediate_columns = ['channel_id', 'text', 'date', 'prev_date', 'description', 'name', 'title']
    categorical_features = ['is_forwarded', 'has_image', 'day', 'month', 'hour']
    textual_features = ['combined_text']
    numerical_features_with_mean = [
        'hour_diff', 'posts_last_30_days', 'post_count', 'user_count',
        'post_count * user_count', 'user_count * user_count',
        'user_count / post_count', 'min_views', 'max_views',
        'mean_views', 'mean_last_10_views'
    ]
    numerical_features_with_zero = ['text_len', 'description_len', 'name_len', 'title_len']
    numerical_features_with_global_mean = ['min_views', 'max_views', 'mean_views', 'mean_last_10_views']
    numerical_features = list(set(numerical_features_with_mean + numerical_features_with_zero))
    target_feature = ['views']
    raw_data_cols = ['channel_id', 'id', 'text', 'date', 'is_forwarded', 'has_image']
    raw_channel_cols = ['channel_id', 'description', 'name', 'title', 'post_count', 'user_count']
    global_mean_views = 14373

    @staticmethod
    @property
    def features_list() -> tuple:
        """ Return tuple of numerical_features, categorical_features, textual_features, target_feature"""
        return (DataProcessor.numerical_features,
                DataProcessor.categorical_features,
                DataProcessor.textual_features,
                DataProcessor.target_feature)

    @staticmethod
    def transform_channel_data(channel_data: DataFrame) -> DataFrame:
        """ Apply transformations to channel data """
        channel_data = (channel_data
                        .withColumn('post_count * user_count', col('post_count') * col('user_count'))
                        .withColumn('user_count * user_count', col('user_count') * col('user_count'))
                        .withColumn('user_count / post_count', col('user_count') / (1 + col('post_count')))
                        .withColumn('description_len', length('description'))
                        .withColumn('name_len', length('name'))
                        .withColumn('title_len', length('title'))
                       )

        return channel_data

    @staticmethod
    def compute_channel_statistics(train_data: DataFrame) -> DataFrame:
        """ Compute statistics for channels in the training data """
        return train_data.select('channel_id', 'views').groupBy('channel_id').agg(
            Fmin('views').alias('min_views'),
            Fmax('views').alias('max_views'),
            Fmean('views').alias('mean_views')
        ).drop('views')

    @staticmethod
    def compute_rolling_avg_views(train_data: DataFrame) -> DataFrame:
        """ Compute rolling average views of recent posts per channel """
        windowSpec = Window.partitionBy("channel_id").orderBy(col("date").desc()).rowsBetween(Window.currentRow + 1, Window.currentRow + 11)
        return train_data.withColumn("rolling_avg_views", Fmean("views").over(windowSpec))

    @staticmethod
    def compute_mean_last_10_views(train_data: DataFrame) -> DataFrame:
        """ Compute mean views of the last 10 posts for each channel """
        windowSpec = Window.partitionBy("channel_id").orderBy(col("date").desc())
        ranked_data = train_data.withColumn("rank", F.rank().over(windowSpec))
        return ranked_data.filter((col("rank") > 1) & (col("rank") <= 11)).groupBy("channel_id").agg(Fmean("views").alias("mean_last_10_views"))

    @staticmethod
    def preprocess_data(train_data: DataFrame, test_data: DataFrame, channel_data: DataFrame) -> tuple:
        """ Preprocess the train and test data """
        train_data = train_data.select(*DataProcessor.raw_data_cols, *DataProcessor.target_feature)
        channel_data = channel_data.select(*DataProcessor.raw_channel_cols)
        test_data = test_data.select(*DataProcessor.raw_data_cols)

        channel_data = DataProcessor.transform_channel_data(channel_data)
        channel_statistics = DataProcessor.compute_channel_statistics(train_data)
        channel_data = channel_data.join(channel_statistics, on='channel_id', how='left')


        train_data = DataProcessor.enrich_data(train_data, channel_data)
        test_data = DataProcessor.enrich_data(test_data, channel_data)

        # Add rolling average views to train data
        train_data = DataProcessor.compute_rolling_avg_views(train_data)
        train_data = train_data.withColumnRenamed("rolling_avg_views", "mean_last_10_views")

        # Add rolling average views to test data
        mean_last_10_views_df = DataProcessor.compute_mean_last_10_views(train_data)
        test_data = test_data.join(mean_last_10_views_df, 'channel_id', 'left')

        # Fill missing values
        train_data = DataProcessor.fill_missing_values(train_data)
        test_data = DataProcessor.fill_missing_values(test_data)

        train_data = DataProcessor.drop_intermediate_columns(train_data)
        test_data = DataProcessor.drop_intermediate_columns(test_data)

        return train_data, test_data, channel_data

    @staticmethod
    def enrich_data(data: DataFrame, channel_data: DataFrame) -> DataFrame:
        """ Enrich data with various transformations """
        data = (data
                .withColumn('day', dayofmonth('date'))
                .withColumn('month', hour('date'))
                .withColumn('hour', month('date'))
                .withColumn("is_forwarded", when(data["is_forwarded"] == 'f', 0).otherwise(1))
                .withColumn("has_image", when(data["has_image"] == 'f', 0).otherwise(1))
                .withColumn('date', col('date').cast('timestamp'))
               )

        windowSpec = Window.partitionBy("channel_id").orderBy("date")
        data = data.withColumn("prev_date", lag("date").over(windowSpec))
        data = data.withColumn('hour_diff', (col('date') - col('prev_date')).cast("long") / 3600)

        windowSpec_last_30 = Window.partitionBy("channel_id").orderBy(col("date").cast("long")).rangeBetween(-86400*31, -86400*1)
        data = data.withColumn("posts_last_30_days", count(col("*")).over(windowSpec_last_30))

        data = data.withColumn('text_len', length('text'))
        data = data.join(channel_data, on='channel_id', how='left')

        data = data.withColumn("combined_text", concat_ws(" ", col("text"), col("description"), col("name"), col("title")))
        data = TextColProcessor.remove_text_artifacts(data, "combined_text")

        return data

    @classmethod
    def drop_intermediate_columns(cls, data: DataFrame) -> DataFrame:
        """ Drop intermediate columns. Utility for train_data and test_data """
        return data.select([col for col in data.columns if col not in DataProcessor.intermediate_columns])

    @staticmethod
    def fill_missing_values(data: DataFrame) -> DataFrame:
        """ Fill missing values in the DataFrame """
        # Fill categorical features with -1
        for feature in DataProcessor.categorical_features:
            data = data.withColumn(feature, when(col(feature).isNull(), lit(-1)).otherwise(col(feature)))

        # Fill textual features with empty string
        for feature in DataProcessor.textual_features:
            data = data.withColumn(feature, when(col(feature).isNull(), lit('')).otherwise(col(feature)))

        # Fill numerical features with mean
        mean_values = data.select([
            Fmean(col(feature)).alias(feature) for feature in DataProcessor.numerical_features_with_mean
        ]).first().asDict()

        for feature, mean_value in mean_values.items():
            data = data.withColumn(feature, when(col(feature).isNull(), lit(mean_value)).otherwise(col(feature)))

        # Fill specific numerical features with 0
        for feature in DataProcessor.numerical_features_with_zero:
            data = data.withColumn(feature, when(col(feature).isNull(), lit(0)).otherwise(col(feature)))

        for feature in DataProcessor.numerical_features_with_global_mean:
            data = data.withColumn(feature, when(col(feature).isNull(), lit(DataProcessor.global_mean_views)).otherwise(col(feature)))

        return data



# load data

In [19]:
train_data = ss.read.orc('hw_data/posts_train.orc')
test_data = ss.read.orc('hw_data/posts_test.orc')
channel_data = ss.read.orc('hw_data/channels_orc')

# analysis

In [ ]:
# test_data.limit(5).show()

# train_data.limit(5).show()

# channel_data.limit(5).show()

# channel_data.select("is_private").groupBy("is_private").count().show() #:)

# train_data.groupBy("has_image").count().show()

# train_data.groupBy("has_image").mean('views').show()

# train_data.filter(train_data["forwarded_id"].isNull()).count() / train_data.count()

# train_data.select('date').describe().show()

# test_data.select('date').describe().show()

# make features

In [38]:
data_processor = DataProcessor()
train_data_prep, test_data_prep, channel_data_prep = data_processor.preprocess_data(train_data, test_data, channel_data)
train_data_prep, test_data_prep, channel_data_prep = train_data_prep.cache(), test_data_prep.cache(), channel_data_prep.cache()

24/06/15 17:53:39 WARN CacheManager: Asked to cache already cached data.        


# save datasets

In [39]:
train_data_prep.show(5, vertical=True)

-RECORD 0---------------------------------------
 id                      | 48093010             
 is_forwarded            | 0                    
 has_image               | 0                    
 views                   | 3026                 
 day                     | 5                    
 month                   | 14                   
 hour                    | 12                   
 hour_diff               | 84.535               
 posts_last_30_days      | 13.0                 
 text_len                | 0                    
 post_count              | 791.0                
 user_count              | 4868.0               
 post_count * user_count | 3850588.0            
 user_count * user_count | 2.3697424E7          
 user_count / post_count | 6.146464646464646    
 description_len         | 161                  
 name_len                | 18                   
 title_len               | 24                   
 min_views               | 1127.0               
 max_views          

In [40]:
test_data_prep.show(5, vertical=True)

-RECORD 0---------------------------------------
 id                      | 57193448             
 is_forwarded            | 0                    
 has_image               | 0                    
 day                     | 2                    
 month                   | 16                   
 hour                    | 1                    
 hour_diff               | 4.0206882918230376   
 posts_last_30_days      | 0.0                  
 text_len                | 0                    
 post_count              | 791.0                
 user_count              | 4868.0               
 post_count * user_count | 3850588.0            
 user_count * user_count | 2.3697424E7          
 user_count / post_count | 6.146464646464646    
 description_len         | 161                  
 name_len                | 18                   
 title_len               | 24                   
 min_views               | 1127.0               
 max_views               | 87914.0              
 mean_views         

In [ ]:
# final features
 # is_forwarded            | 0                    
 # has_image               | 0                    
 # day                     | 2                    # date
 # month                   | 16                   # date
 # hour                    | 1                    # date
 # hour_diff               | 0.008611111111111111 # partition
 # posts_last_30_days      | 0.0                  # partition
 # text_len                | 0                    #text feat
 # post_count              | 791.0                #channel metadata
 # user_count              | 4868.0               #channel metadata
 # post_count * user_count | 3850588.0            #channel metadata
 # user_count * user_count | 2.3697424E7          #channel metadata
 # user_count / post_count | 6.146464646464646    #channel metadata
 # description_len         | 161                  #channel metadata text
 # name_len                | 18                   #channel metadata text
 # title_len               | 24                   #channel metadata text
 # min_views               | 1127.0               
 # max_views               | 87914.0              
 # mean_views              | 5523.499367888749    
 # combined_text           | Нужна реклама туп...  #text concated and cleaned with udf for catboost!
 # mean_last_10_views      | 5279.6                # partition

In [42]:
train_data_prep.write.csv('data_prep/train_csv', mode='overwrite', header=True)
test_data_prep.write.csv('data_prep/test_csv', mode='overwrite', header=True)

# load features to pandas
you also can use .toPandas()

In [9]:
# %cd 'drive/MyDrive/BigDataProject'

/content/drive/MyDrive/BigDataProject


In [7]:
import subprocess
import glob
import os
import shutil
import pandas as pd

def load_and_merge_csv(path, **kwargs):
    dfs = []
    for g in glob.glob(os.path.join(path, '*.csv')):
        dfs.append(pd.read_csv(g, **kwargs))
    res = pd.concat(dfs)
    res = res.set_index('id')
    return res

In [8]:
!ls

'BigData.tg_views_solution.Novitskij O.ipynb'   ports.log   train_csv
 onstart.sh				        test_csv


In [9]:
trainXY_pd = load_and_merge_csv('train_csv')
testX_pd = load_and_merge_csv('test_csv')

In [10]:
# trainvalXY = ss.read.csv('drive/MyDrive/BigDataProject/data_prep/train_csv', header=True)
# testX = ss.read.csv('drive/MyDrive/BigDataProject/data_prep/test_csv', header=True)
# trainvalXY = ss.read.csv('./data_prep/train_csv', header=True)
# testX = ss.read.csv('./data_prep/test_csv', header=True)
# Ycol = 'views'
# trainvalXY = trainvalXY.withColumn(Ycol, log(col(Ycol) + lit(100)))
# trainXY, valXY = trainvalXY.randomSplit([0.7, 0.3], 42)
# trainXY_pd = trainXY.toPandas()
# valXY_pd = valXY.toPandas()
# testX_pd = testX.toPandas()

In [16]:
Ycol = DataProcessor.target_feature[0]
trainX_pd, valX_pd, trainY_pd, valY_pd = train_test_split(trainXY_pd.drop(columns=[Ycol]),
                                                          trainXY_pd[[Ycol]],
                                                          random_state=42)
trainY_pd = np.log(100 + trainY_pd)
valY_pd = np.log(100 + valY_pd)

In [17]:
categorical_features = DataProcessor.categorical_features
text_features = DataProcessor.textual_features

# Создание пула данных для CatBoost
train_pool = Pool(data=trainX_pd,
                  label=trainY_pd,
                  cat_features=categorical_features,
                  text_features=text_features)

val_pool = Pool(data=valX_pd,
                  label=valY_pd,
                  cat_features=categorical_features,
                  text_features=text_features)

test_pool = Pool(data=testX_pd,
                 cat_features=categorical_features,
                 text_features=text_features)

# Определение и обучение модели
model = CatBoostRegressor(
    verbose=1,
    # save_snapshot=True,
    task_type='GPU',
    # allow_writing_files=True
)

# Обучение модели
model.fit(train_pool, eval_set=val_pool, 
          # use_best_model=True
         )

# Прогнозирование на тестовой выборке
prediction = model.predict(test_pool)

# Оценка модели
# mse = mean_squared_error(y_test, predictions)
# print(f'Ошибка среднеквадратическая (MSE): {mse}')

# Печать прогнозов
# print(f'Прогнозы: {predictions}')

Learning rate set to 0.15249
0:	learn: 1.3332557	test: 1.3332514	best: 1.3332514 (0)	total: 49.3ms	remaining: 49.2s
1:	learn: 1.2152739	test: 1.2155054	best: 1.2155054 (1)	total: 95.5ms	remaining: 47.6s
2:	learn: 1.1197587	test: 1.1202425	best: 1.1202425 (2)	total: 150ms	remaining: 49.8s
3:	learn: 1.0440850	test: 1.0448051	best: 1.0448051 (3)	total: 196ms	remaining: 48.8s
4:	learn: 0.9840181	test: 0.9849400	best: 0.9849400 (4)	total: 240ms	remaining: 47.8s
5:	learn: 0.9364276	test: 0.9375206	best: 0.9375206 (5)	total: 294ms	remaining: 48.7s
6:	learn: 0.8994393	test: 0.9007068	best: 0.9007068 (6)	total: 338ms	remaining: 47.9s
7:	learn: 0.8702640	test: 0.8715995	best: 0.8715995 (7)	total: 381ms	remaining: 47.2s
8:	learn: 0.8485338	test: 0.8499899	best: 0.8499899 (8)	total: 424ms	remaining: 46.7s
9:	learn: 0.8307853	test: 0.8322914	best: 0.8322914 (9)	total: 476ms	remaining: 47.1s
10:	learn: 0.8170386	test: 0.8186673	best: 0.8186673 (10)	total: 520ms	remaining: 46.8s
11:	learn: 0.8053259	

In [29]:
import catboost

w = catboost.MetricVisualizer('catboost_info')
w.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

# train your model and predict test

In [19]:
prediction.shape

(244386,)

In [20]:
assert prediction.shape == (244386,)

# submit

In [21]:
! curl -H 'Cache-Control: no-cache' https://raw.githubusercontent.com/mike0sv/lsml_submit_server/2024/src/client.py -o client.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1899  100  1899    0     0   9689      0 --:--:-- --:--:-- --:--:--  9738


In [22]:
import client

In [27]:
client.make_eval(pd.DataFrame({'views': prediction}, index=testX_pd.index), final=True)

{'data': {'mape': 6.943677375983398,
  'mean_absolute_error': 0.5525682475573108,
  'mean_squared_error': 0.8931411136397162,
  'rmse': 0.9450614337913257,
  'rmspe': 13.138363225495914},
 'ok': True}

In [28]:
client.check_results()

{'2024-06-15': [{'baseline_beaten': True,
   'is_final': False,
   'metrics': {'mape': 6.943677375983398,
    'mean_absolute_error': 0.5525682475573108,
    'mean_squared_error': 0.8931411136397162,
    'rmse': 0.9450614337913257,
    'rmspe': 13.138363225495914}},
  {'baseline_beaten': True,
   'is_final': True,
   'metrics': {'mape': 6.943677375983398,
    'mean_absolute_error': 0.5525682475573108,
    'mean_squared_error': 0.8931411136397162,
    'rmse': 0.9450614337913257,
    'rmspe': 13.138363225495914}}]}

In [ ]:
# baseline
# {
# 	'mape': 15.707128974856676,
# 	'mean_absolute_error': 1.219070382113261,
# 	'mean_squared_error': 2.4324378881170055,
# 	'rmse': 1.5596274837655963,
# 	'rmspe': 23.50065988751091
# }